In [1]:
import os
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.aboutdepart

In [14]:
departments = os.listdir('school')

In [127]:
def getAuthorID(dataframe):
    rows=[]
    try:
        for i in dataframe['Author Ids'][0].split(';'):
            for row in dataframe['Author Ids']:
                rows.append(row.split(';'))
            for i in dataframe['Author Ids'][0].split(';'):
                count=0
                for row in rows:
                    if i in row:
                        count += 1
                if count == len(dataframe['Author Ids']):
                    return i
    except:
        for i in dataframe['Author(s) ID'][0].split(';'):
            for row in dataframe['Author(s) ID']:
                rows.append(row.split())
            for i in dataframe['Author(s) ID'][0].split(';'):
                count=0
                for row in rows:
                    if i in row:
                        count += 1
                if count == len(dataframe['Author(s) ID']):
                    return i
def getAuthor(dataframe):
    results={}
    rows=[]
    for i in dataframe['Authors'][0].split(';'):
        for row in dataframe['Authors']:
            rows.append(row.split(','))
        for i in dataframe['Authors'][0].split(','):
            count=0
            for row in rows:
                if i in row:
                    count += 1
            results[count] = i
    return results[max(list(results.keys()))]

In [137]:
db.professors.drop()
for department in departments:
    professors = os.listdir('school/'+department)
    for professor in professors:
        document={}
        if '.csv' not in professor:
            continue
        dataframe = pd.read_csv('school/'+department+"/"+professor)
        for k in dataframe.keys():
            column_data=[]
            for idx, r in enumerate(dataframe[k]):
                if r == '[No abstract available]':
                    r = dataframe['Title'][idx]
                column_data.append(str(r))
            document[k.replace('.','-')] = column_data
        try:
            document['Author ID'] = getAuthorID(dataframe)
            document['Author'] = getAuthor(dataframe)
            db.professors.insert_one(document)
        except:
            print(professor)

In [11]:
count = 0
for doc in db.profdata.find():
    try:
        res = db.professors.find_one({"Author ID":doc['id']})
        author_id = doc.pop('id')
        doc.pop('_id')
        db.professors.update_one({"Author ID":author_id},{"$set":doc})
    except:
        count += 1
    if res == None:
        count += 1
print(count)

176


In [12]:
db.professors.count()

C:\Users\thaloo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


270

In [13]:
for doc in db.professors.find():
    print(doc.keys())
    break

dict_keys(['_id', 'Authors', 'Author Ids', 'Title', 'Year', 'Source title', 'Volume', 'Issue', 'Art- No-', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract', 'Author Keywords', 'Document Type', 'Access Type', 'Source', 'EID', 'Author ID', 'Author', 'Lab', 'ResearchInterest', 'Website', 'department', 'Name'])


In [7]:
for doc in db.profdata.find():
    print(doc)

{'_id': ObjectId('5be82b9320c66c2c9ea6044f'), 'id': '55614074100', 'department': 'School of Mechanical, Aerospace and Nuclear Engineering(기계항공및원자력공학부)', 'ResearchInterest': 'Nuclear Fuel Performance Experiments & Modeling, Radiation Interactions with Matter, Thermophysical Investigation of Nuclear Materials, Nuclear Non-Proliferation Technology       ', 'Lab': 'UNIST RAdioactive NUclear Materials (URANUM) Laboratory', 'Website': 'http://uranum.unist.ac.kr'}
{'_id': ObjectId('5be82b9420c66c2c9ea60450'), 'id': '7004730960', 'department': 'School of Life Sciences(생명과학부)', 'ResearchInterest': 'Biology, Cancer Biology       ', 'Lab': 'School of Life Science', 'Website': ''}
{'_id': ObjectId('5be82b9620c66c2c9ea60451'), 'id': '7202344861', 'department': 'School of Energy and Chemical Engineering(에너지및화학공학부)', 'ResearchInterest': 'Nanoparticle catalytsts, catalytic activity, selectivity, and stability, strong metal-support interactions, biofuel conversion       ', 'Lab': 'Advanced Nanocatalysi